En esta tarea aprenderás un par de métodos eficientes para hacer polígonos de Voronoi. Es importante que al menos uno de los métodos eficientes lo programes correctamente, pues usarás tu código para producir polígonos en varias tareas futuras. 


# Polígonos de Voronoi

Imaginemos que pasas a ser director de la oficina central de correos de una ciudad y tienes un mapa de todas las oficinas de correos de tu ciudad. Tu primera labor consiste en determinar a qué oficina enviarás cada una de las cartas que recibas para que los carteros de esas oficinas entreguen las cartas a sus destinatarios. Entonces, quieres dividir la ciudad en regiones, de tal forma que el tiempo que se haga un cartero saliendo de su oficina hasta cualquier hogar que le corresponda, sea menor que si ese mismo cartero saliera desde cualquier otra oficina en la ciudad. 

Esta es la idea de fondo sobre los polígonos de Voronoi, construir regiones, tales que saliendo desde los centros de cada celda, la distancia (o tiempo) hasta cualquier punto de la celda de Voronoi correspondiente, sea menor que si se saliera desde cualquier otro centro. Así, para construir un polígono de Voronoi, necesitamos un conjunto de puntos, los cuales serán los centros de nuestros polígonos. 

# Fuerza Bruta

Hay muchas formas de construir por fuerza bruta una región de Voronoi. Comencemos por construir una región aproximada usando lo que ya conocemos, un convex-hull. 

## Método aproximado y con fuerza bruta

Antes, demostremos que una celda de Voronoi siempre será una región convexa. Para esto supongamos que no, que existe una celda que no es convexa, esto es hay un par de puntos en la región que al unirlos por un segmento recto, cortan la frontera de la celda, es decir, cruzan a otra región de Voronoi. Eso implica que la distancia de un punto, en esa parte del segmento donde cruzan a otra celda, al centro de Voronoi más cercano, sería al mismo centro cuya celda no es convexa, pero esto es una contradicción, pues el punto inicial pertenece a otra celda y por lo tanto, la distancia menor a un centro debería ser a su propio centro. Es decir, todos los polígonos de Voronoi son convexos. 

La idea entonces para aproximar los polígonos de Voronoi, es poner puntos aleatorios en el plano y subdividir en regiones determinando cuál de los centros les queda más cerca. O sea, necesitamos hacer una lista de listas, con tantos elementos como centros hayamos puesto. Finalmente, a cada una de esas listas, aplicarles un convexhull, por ejemplo, usando el algoritmo de Graham. 

***Nota: En los algoritmos de fuerza bruta, utiliza pocos centros para hacer tus pruebas, digamos, del orden de 10-20 puntos***

[1] Haz una función que, dado un conjunto de puntos (centros) en el plano, y un punto en particular, encuentre qué centro es el más cercano. 

In [1]:
using Plots
gr()

Plots.GRBackend()

In [8]:
using Base.Iterators

In [2]:
include("../utilidades.jl"); # Algunas funciones a utilizar

In [3]:
function centro_cercano(centros, punto)
    
    distancias = []
    
    for centro in centros
        d = norm(centro - punto)
        push!(distancias, d)
    end
    
    i_min = indmin(distancias)
    centros[i_min]
    
end

centro_cercano (generic function with 1 method)

In [190]:
n_puntos = 10
algunos_centros = [[rand(), rand()] for i in 1:n_puntos]
un_punto = rand(2, 1)
centro_mas_cerano = centro_cercano(algunos_centros, un_punto)


scatter(arr_to_xy(algunos_centros), label="", ratio=:equal, xlims=(0, 1), ylims=(0, 1))
scatter!([un_punto[1]], [un_punto[2]], label="")
scatter!([centro_mas_cerano[1]], [centro_mas_cerano[2]], label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0

[2] Usando la función anterior, haz una función que dado un conjunto de centros y un número n de iteraciones, genere aleatoriamente puntos en el plano (restringidos a una región, por ejemplo un cuadrado de lado L y centrado en $p_c$ dados) y con ello genere listas de puntos clasificados según el centro que se encuentre más cercano. 

[3] Haz una función que usando el problema 2 y la función de Convexhull que hiciste en la tarea pasada, que genere los convexhull de cada una de las listas de puntos. 

[4] Haz varias pruebas y piensa cómo determinar qué región es vecina de cuál otra. Implementa lo que se te haya ocurrido. ¿Qué tan confiable es tu método?¿Qué valores de $n$ requieres para confiar en tu método? ¿Cuál es la complejidad computacional de este algoritmo de fuerza bruta como función de $n$?

In [3]:
function genera_puntos(centros, n_iteraciones, L=1, p_c=[0, 0])
    
    puntos = [[p_c[1] + L*rand(), p_c[2] + L*rand()] for i in 1:n_iteraciones] # Creo los puntos
    puntos_clf = Dict(string(c) => Array[] for c in centros) # Un dic para clasficar por centros los puntos
    
    for punto in puntos
        centro = centro_cercano(centros, punto)
        push!(puntos_clf[string(centro)], punto)
    end
    
    collect(values(puntos_clf))
    
end

genera_puntos (generic function with 3 methods)

In [161]:
function multiple_convex(puntos_clf)
    
    hulls = [] 
    
    for puntos in puntos_clf
        
        if length(puntos) == 0
            continue
        end
        
        push!(hulls, convex_graham(puntos)) # Estoy usando el método de Graham
    end
    
    hulls
    
end

multiple_convex (generic function with 1 method)

In [315]:
n_iteraciones = 3000
n_centros = 40
centros = [[rand(), rand()] for i in 1:n_centros]
puntos_clf = genera_puntos(centros, n_iteraciones);
hulls = multiple_convex(puntos_clf);

p = dibuja_convex(puntos_clf[1], hulls[1])

for i in 2:n_centros
    dibuja_convex(puntos_clf[i], hulls[i], append=true)
end
p

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip1502)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 374.061,211.997 374.002,211.979 373.944,211.96 373.886,211.942 373.827,211.923 373.769,211.905 373.711,211.886 373.652,211.867 373.594,211.849 373.536,211.83 
 373.477,211.812 373.419,211.793 373.36,211.775 373.302,211.756 373.244,211.737 373.185,211.719 373.127,211.7 373.069,211.682 373.01,211.663 372.952,211.645 
 372.894,211.626 372.835,211.608 372.777,211.589 372.719,211.57 372.66,211.552 372.602,211.533 372.544,211.515 372.485,211.496 372.427,211.478 372.369,211.459 
 372.31,211.44 372.252,211.422 372.194,211.403 372.135,211.385 372.077,211.366 372.019,211.348 371.96,211.329 371.902,211.31 371.844,211.292 371.785,211.273 
 371.727,211.255 371.669,211.236 371.61,211.218 371.552,211.199 371.493,211.18 371.435,211.162 371.377,211.143 371.318,211.125 371.26,211.106 371.202,211.088 
 371.143,211.069 371.085,211.05 371.027,211.032 370.968,211.013 370.91,210.995 370.852,210.976 370.793,210.958 370.735,210.939 370.677,210.92 370.618,210.902 
 370.56,210.883 370.502,210.865 370.443,210.846 370.385,210.828 370.327,210.809 370.268,210.79 370.21,210.772 370.152,210.753 370.093,210.735 370.035,210.716 
 369.977,210.698 369.918,210.679 369.86,210.661 369.802,210.642 369.743,210.623 369.685,210.605 369.626,210.586 369.568,210.568 369.51,210.549 369.451,210.531 
 369.393,210.512 369.335,210.493 369.276,210.475 369.218,210.456 369.16,210.438 369.101,210.419 369.043,210.401 368.985,210.382 368.926,210.363 368.868,210.345 
 368.81,210.326 368.751,210.308 368.693,210.289 368.635,210.271 368.576,210.252 368.518,210.233 368.46,210.215 368.401,210.196 368.343,210.178 368.285,210.159 
 
 "/>
<polyline clip-path="url(#clip1502)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 368.285,210.159 368.144,210.022 368.003,209.886 367.862,209.749 367.721,209.612 367.58,209.475 367.44,209.339 367.299,209.202 367.158,209.065 367.017,208.928 
 366.876,208.792 366.735,208.655 366.595,208.518 366.454,208.381 366.313,208.245 366.172,208.108 366.031,207.971 365.89,207.834 365.749,207.698 365.609,207.561 
 365.468,207.424 365.327,207.287 365.186,207.151 365.045,207.014 364.904,206.877 364.764,206.74 364.623,206.604 364.482,206.467 364.341,206.33 364.2,206.193 
 364.059,206.057 363.919,205.92 363.778,205.783 363.637,205.646 363.496,205.51 363.355,205.373 363.214,205.236 363.074,205.099 362.933,204.963 362.792,204.826 
 362.651,204.689 362.51,204.552 362.369,204.416 362.228,204.279 362.088,204.142 361.947,204.005 361.806,203.869 361.665,203.732 361.524,203.595 361.383,203.458 
 361.243,203.322 361.102,203.185 360.961,203.048 360.82,202.911 360.679,202.775 360.538,202.638 360.398,202.501 360.257,202.364 360.116,202.228 359.975,202.091 
 359.834,201.954 359.693,201.817 359.553,201.681 359.412,201.544 359.271,201.407 359.13,201.27 358.989,201.134 358.848,200.997 358.708,200.86 358.567,200.723 
 358.426,200.587 358.285,200.45 358.144,200.313 358.003,200.176 357.862,200.04 357.722,199.903 357.581,199.766 357.44,199.629 357.299,199.493 357.158,199.356 
 357.017,199.219 356.877,199.083 356.736,198.946 356.595,198.809 356.454,198.672 356.313,198.536 356.172,198.399 356.032,198.262 355.891,198.125 355.75,197.989 
 355.609,197.852 355.468,197.715 355.327,197.578 355.187,197.442 355.046,197.305 354.905,197.168 354.764,197.031 354.623,196.895 354.482,196.758 354.342,196.621 
 
 "/>
<polyline clip-path="url(#clip1502)" style="stroke:#ff0000; s

In [ ]:
# Complejidad

for 

## Método exacto con fuerza bruta

Las fronteras entre vecinos de celdas de Voronoi, son puntos que son equidistantes a ambos centros. La región del plano que es equidistante a dos puntos dados es la mediatriz del segmento que une a los puntos. Entonces, un polígono de Voronoi se formará por la intersección de las mediatrices entre los diferentes vecinos de cada centro. El problema es encontrar quienes son dichos vecinos. 

Pensemos en el caso donde restringimos nuestras celdas de Voronoi a una región dada, por ejemplo, un cuadrado de lado dado y centro dado. Entonces, cada una de las mediatrices estará dada por un segmento que divida la región dada en 2 partes, una conteniendo a un centro, la otra conteniendo a otro. La celda de Voronoi asociada a un centro $i$ tendrá la propiedad de que cualquier mediatriz que no sea parte de la celda, no intersectará a esta. Entonces, podemos generar el siguiente algoritmo: 

1. Producimos una mediatriz entre un centro $i$ cualquiera y cualquier otro en la lista $j$ y lo intersectamos con la región $C$ a la cual nos limitamos para tener 2 puntos (llamaremos de ahora en adelante la mediatriz $(i,j)$ a este segmento). Agregamos este segmento a la lista de la celda de Voronoi del centro $i$, llamémosle $V(i)$.
- Producimos otra mediatriz $(i,k)$ con $k \neq j$ y $k \neq i$ . Revisamos si intersecta uno o dos de los segmentos  $V(i)$ (inicialmente con solo un segmento). Si intersecta 2 tendremos ahora un semgento más en la lista, con cada uno de sus extremos los puntos las intersecciones, además, cada uno de los segmentos intersectados se reducirán de tamaño (proceso que llamaremos reducción del segmento), teniendo como uno de sus extremos el punto de intersección. Para decidir cuál de los extremos de cada segmento se queda como parte del segmento reducido, dividimos $C$ en dos regiones usando la recta que pasa por el nuevo segmento producido. El extremo que se quedará como parte del segmento reducido será el que se encuentre en la misma región que el centro $i$. Si intersecta un solo punto, agregamos la mediatriz y después reducimos, tanto esa mediatriz, como el segmento que fue intersectado. Finalmente, si nuestra mediatriz no intersecta ningún segmento, observamos que esta mediatriz divide nuevamente en 2 subregiones la región a la que estamos constriñidos (el cuadrado) revisamos si uno de los puntos (cualquiera) de la lista de la celda de Voronoi se encuentra en la misma subregión del centro $i$ o no. Si se encuentra en la misma subregión, pasamos al siguiente paso, sino, borramos toda la lista y agregamos la mediatriz. 
- Repetimos el paso 2, hasta que se hayan revisado todos los centros $j\neq i$. 
- Regresamos al paso 1, hasta que $i$ haya recorrido todos los centros. 

[5] Haz una función que dados dos segmentos determine si hay intersección y si la hay, regrese el punto de intersección.



In [3]:
function intersecta(p1, p2, p3, p4)
    
    if (ccw(p1, p3, p4) != ccw(p2, p3, p4)) & (ccw(p1, p2, p3) != ccw(p1, p2, p4))
        
        # Para encontrar la intersección hay que resolver un sist. de ec. derivado de la forma paramétrica de los segm.
        x1, y1 = p1[1], p1[2]
        x2, y2 = p2[1], p2[2]
        x3, y3 = p3[1], p3[2]
        x4, y4 = p4[1], p4[2]
        
        # Aquí def. el sist. a resolver
        A = [x2-x1 -(x4-x3); y2-y1 -(y4-y3)]
        b = [x3-x1; y3-y1]
        s0, t0 = A^-1*b
        
        # Ahora evalúo en la forma paramétrica
        if s0 >= 0
            x0, y0 = [x1 + s0*(x2 - x1), y1 + s0*(y2 - y1)]
            return [x0, y0]
        elseif (s0 < 0) & (t0 >= 0)
            x0, y0 = [x3 + t0*(x4 - x3), y3 + t0*(y4 - y3)]
            return [x0, y0]
        else
            return error
        end
        
    else
        return false
    end
    
end

intersecta (generic function with 1 method)

[6] Haz una función que dados 2 puntos y una región convexa (por ejemplo un cuadrado), obtenga los 2 puntos de intersección entre la mediatriz de los puntos y la región. 

[7] Haz una función que dada una región convexa (por ejemplo, un cuadrado), una mediatriz (los dos puntos de intersección) y dos puntos dados, determine si ambos puntos están en la misma región, o no. 

In [4]:
function mediatriz(p1, p2)
    
    x1, y1 = p1
    x2, y2 = p2
    m = (y2 - y1) / (x2 - x1)
    
    m_ort = -1/m
    punto_medio = [(x1 + x2) / 2, (y1 + y2) / 2]
    y(x) = m_ort * (x - punto_medio[1]) + punto_medio[2] # Ec. de la mediatriz
    
    y
    
end

mediatriz (generic function with 1 method)

In [5]:
function intersecta_region(p1, p2; convex_reg=[[0,0], [0,1], [1,1], [1,0]])
    
    # Los vértices tienen que estar en orden, la reg. se recorre a la derecha
    segmentos = [[convex_reg[i], convex_reg[i+1]] for i in 1:(length(convex_reg)-1)]
    push!(segmentos, [convex_reg[1], convex_reg[end]])
    
    inters = []
    seg_inters = []
    δ = .2
    #x_r = [convex_reg[1][1], convex_reg[end][1]]
    x_r = [0, 1]
    y_bis = mediatriz(p1, p2)
    x1 = x_r[1] - δ
    x2 = x_r[2] + δ
    p1, p2 = [x1, y_bis(x1)], [x2, y_bis(x2)]
    
    for seg in segmentos
        p3, p4 = seg
        intersecc = intersecta(p1, p2, p3, p4)
        
        if intersecc != false
            push!(inters, intersecc)
            push!(seg_inters, seg)
        end
    
    end
    
    
    inters, seg_inters
    
end

intersecta_region (generic function with 1 method)

In [6]:
function recta(p1, p2)
    
    x -> ((p2[2] - p1[2]) / (p2[1] - p1[1]) * (x - p1[1])) + p1[2]
    
end

recta (generic function with 1 method)

[8] Implementa el algoritmo de fuerza bruta para los polígonos de Voronoi. 

[9] Haz algunas pruebas, graficando los polígonos. 

[10] Mide la complejidad computacional del algoritmo.

[11] ¿Podrías generalizar este algoritmo a 3D? ¿Qué se requiere para generalizarlo?

In [7]:
function misma_region2(P1, P2, y)
    
    p1 = [0, y(0)]
    p2 = [1, y(1)]
    x1, y1 = p1
    x2, y2 = p2
    
    s(p) = (x2 - x1)*(p[2] - y1) - (y2 - y1)*(p[1] - x1)
    
    sigP = sign(s(P1))
    sigC = sign(s(P2))
    
    if sigP == sigC
        return true
    elseif (sigP == 0 ) | (sigC == 0)
        return true
    else
        return false
    end
    
end

misma_region2 (generic function with 1 method)

In [113]:
p1 = [rand(), rand()]
p2 = [rand(), rand()]
scatter([p1[1], p2[1]], [p1[2], p2[2]], xlims=(0, 1), ylims=(0,1), labels="")
print(misma_region2(p1, p2, x->x))
plot!(x -> x)


false

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 y2

In [8]:
function reduce_segmento2(seg1, seg2, p_i, centro)
    
    # Va a reducir al segmento 1 mediante la recta que pasa por el segmento 2
    p11, p12 = seg1
    p21, p22 = seg2
    x1, y1 = p21
    x2, y2 = p22
    y(x) = ((y2 - y1) / (x2 - x1) * (x - x1)) + y1
    #p_i = intersecta(seg1..., seg2...)
    
    if misma_region2(p11, centro, y)
        seg1 = [p11, p_i]
    elseif misma_region2(p12, centro, y)
        seg1 = [p12, p_i]
    else
        return false
    end
        
    seg1
        
end

reduce_segmento2 (generic function with 1 method)

In [13]:
x = linspace(0, 1, 100)
plot(x, x, ratio=:equal, label="", ylims=(0,1))
plot!(x, -x + 1, label="")
plot!(x, 4*x, label="")
scatter!([.25], [.25], label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip2402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 124.366,375.869 128.043,372.191 131.721,368.514 135.398,364.837 139.075,361.159 142.753,357.482 146.43,353.805 150.107,350.127 153.785,346.45 157.462,342.773 
 161.139,339.095 164.817,335.418 168.494,331.74 172.171,328.063 175.849,324.386 179.526,320.708 183.203,317.031 186.881,313.354 190.558,309.676 194.236,305.999 
 197.913,302.322 201.59,298.644 205.268,294.967 208.945,291.29 212.622,287.612 216.3,283.935 219.977,280.258 223.654,276.58 227.332,272.903 231.009,269.226 
 234.686,265.548 238.364,261.871 242.041,258.193 245.718,254.516 249.396,250.839 253.073,247.161 256.751,243.484 260.428,239.807 264.105,236.129 267.783,232.452 
 271.46,228.775 275.137,225.097 278.815,221.42 282.492,217.743 286.169,214.065 289.847,210.388 293.524,206.711 297.201,203.033 300.879,199.356 304.556,195.679 
 308.233,192.001 311.911,188.324 315.588,184.646 319.265,180.969 322.943,177.292 326.62,173.614 330.298,169.937 333.975,166.26 337.652,162.582 341.33,158.905 
 345.007,155.228 348.684,151.55 352.362,147.873 356.039,144.196 359.716,140.518 363.394,136.841 367.071,133.164 370.748,129.486 374.426,125.809 378.103,122.132 
 381.78,118.454 385.458,114.777 389.135,111.099 392.812,107.422 396.49,103.745 400.167,100.067 403.845,96.3901 407.522,92.7127 411.199,89.0354 414.877,85.358 
 418.554,81.6807 422.231,78.0033 425.909,74.326 429.586,70.6486 433.263,66.9713 436.941,63.2939 440.618,59.6166 444.295,55.9392 447.973,52.2619 451.65,48.5845 
 455.327,44.9072 459.005,41.2298 462.682,37.5525 466.359,33.8751 470.037,30.1978 473.714,26.5204 477.392,22.8431 481.069,19.1657 484.746,15.4884 488.424,11.811 
 
 "/>
<polyline clip-path="url(#clip2402)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 124.366,11.811 128.043,15.4884 131.721,19.1657 135.398,22.8431 139.075,26.5204 142.753,30.1978 146.43,33.8751 150.107,37.5525 153.785,41.2298 157.462,44.9072 
 161.139,48.5845 164.817,52.2619 168.494,55.9392 172.171,59.6166 175.849,63.2939 179.526,66.9713 183.203,70.6486 186.881,74.326 190.558,78.0033 194.236,81.6807 
 197.913,85.358 201.59,89.0354 205.268,92.7127 208.945,96.3901 212.622,100.067 216.3,103.745 219.977,107.422 223.654,111.099 227.332,114.777 231.009,118.454 
 234.686,122.132 238.364,125.809 242.041,129.486 245.718,133.164 249.396,136.841 253.073,140.518 256.751,144.196 260.428,147.873 264.105,151.55 267.783,155.228 
 271.46,158.905 275.137,162.582 278.815,166.26 282.492,169.937 286.169,173.614 289.847,177.292 293.524,180.969 297.201,184.646 300.879,188.324 304.556,192.001 
 308.233,195.679 311.911,199.356 315.588,203.033 319.265,206.711 322.943,210.388 326.62,214.065 330.298,217.743 333.975,221.42 337.652,225.097 341.33,228.775 
 345.007,232.452 348.684,236.129 352.362,239.807 356.039,243.484 359.716,247.161 363.394,250.839 367.071,254.516 370.748,258.193 374.426,261.871 378.103,265.548 
 381.78,269.226 385.458,272.903 389.135,276.58 392.812,280.258 396.49,283.935 400.167,287.612 403.845,291.29 407.522,294.967 411.199,298.644 414.877,302.322 
 418.554,305.999 422.231,309.676 425.909,313.354 429.586,317.031 433.263,320.708 436.941,324.386 440.618,328.063 444.295,331.74 447.973,335.418 451.65,339.095 
 455.327,342.773 459.005,346.45 462.682,350.127 466.359,353.805 470.037,357.482 473.714,361.159 477.392,364.837 481.069,368.514 484.746,372.191 488.424,375.869 
 
 "/>
<polyline clip-path="url(#clip2402)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 124.366,375.869 128.043,361.159 131.721,346.45 135.398,331.74 139.075,317.031 142.753,302.322 146.43,287.612 150.107,272.903 153.785,258.193 157.462,243.484 
 161.139,228.775 164.817,214.065 168.494,199.35

## Aquí sigue la parte que parece que va a funcionar

In [10]:
function voronoi_final(centros, reg=[[0,0], [0,1], [1,1], [1,0]])

    n = length(centros)
    celdas_voronoi = Dict(string(c) => Array[] for c in 1:n)

    for i in 1:n
        celdas = []
        for j in 1:n

            i == j ? continue: nothing
            ci = centros[i]
            cj = centros[j]


            bis, _ = intersecta_region(ci, cj, convex_reg=reg)

            if length(celdas) == 0
                push!(celdas, bis)
                continue
            end

            aux = false
            seg_idx = []

            for (idx, seg) in enumerate(celdas)

                pi = intersecta(bis..., seg...)
                pi == false ? continue: nothing
                push!(seg_idx, idx)
                bis = reduce_segmento2(bis, seg, pi, ci)
                celdas[idx] = reduce_segmento2(seg, bis, pi, ci)

                aux = true
            end

            push!(celdas, bis)

            # Elimina puntos
            aux_celdas = []
            if aux
                push!(aux_celdas, celdas[seg_idx]...)
                y = [recta(celdas[i]...) for i in seg_idx]
                push!(y, recta(bis...))
                regiones(p) = [misma_region2(p, ci, yi) for yi in y]

                for (idx, seg) in enumerate(celdas)
                    any(idx .== seg_idx)? continue: nothing
                    all(regiones(seg[1])) == false ? continue: push!(aux_celdas, seg)
                end
                celdas = aux_celdas
            else
                misma_region2(celdas[1][1], ci, recta(bis...)) ? deleteat!(celdas, length(celdas)): celdas = [bis]
            end
        end
        celdas_voronoi[string(i)] = celdas
    end

    celdas_voronoi
end

voronoi_final (generic function with 2 methods)

In [39]:
a = [1,2,3]
shift!(a)

1

In [138]:
a = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [139]:
delete!(a)

LoadError: [91mMethodError: no method matching delete!(::Array{Int64,1})[0m
Closest candidates are:
  delete!([91m::IntSet[39m, [91m::Integer[39m) at intset.jl:83
  delete!([91m::ObjectIdDict[39m, [91m::ANY[39m) at associative.jl:450
  delete!([91m::Base.EnvHash[39m, [91m::AbstractString[39m) at env.jl:83
  ...[39m

In [140]:
function voronoi_f(centros)
    
    n = length(centros)
    celdas_voronoi = Dict(string(c) => Array[] for c in 1:n)
    
    for i in 1:n
        celdas = []
        for j in 1:n
            
            i == j ? continue: nothing
            ci = centros[i]
            cj = centros[j]
            
            bis, _ = intersecta_region(ci, cj)
            
                        
            if length(celdas) == 0
                push!(celdas, bis)
                continue
            end
            
            

            aux = false
            aux_celdas = []
            no_int = []
            
            if length(celdas) > 0
                while !isempty(celdas)
                    seg = shift!(celdas)
                    pi = intersecta(bis..., seg...)
                    if pi != false
                        bis = reduce_segmento2(bis, seg, pi, ci)
                        seg_red = reduce_segmento2(seg, bis, pi, ci)
                        push!(aux_celdas, seg_red)
                        aux = true
                    else
                        push!(no_int, seg)
                    end
                end
            end
            

            
            if aux
                push!(aux_celdas, bis)  
                celdas = aux_celdas
            else
                push!(no_int, bis)
                un_punto = no_int[1][1]
                y = recta(bis...)
                misma_region2(un_punto, ci, y) ? deleteat!(no_int, length(no_int)): no_int = [bis]
                celdas = no_int
            end
            
            celdas = aux_celdas
            
        end
        
        celdas_voronoi[string(i)] = celdas
    end
    
    celdas_voronoi
    
end         

voronoi_f (generic function with 1 method)

In [10]:
function dibuja_voronoi(centros, celdas_voronoi; leg=false)

    p = plot(xlims=(0,1.5), ylims=(0,1.5), ratio=:equal)

    for (i, c) in enumerate(centros)

        if leg == true
            scatter!([c[1]], [c[2]], label="C$i")
        else
            scatter!([c[1]], [c[2]], label="")
        end
    end

    for celda in values(celdas_voronoi)
        for seg in celda
            x1, y1 = seg[1]
            x2, y2 = seg[2]
            x = linspace(x1, x2, 100)
            yy(x) = ((y2 - y1) / (x2 - x1) * (x - x1)) + y1
            plot!(x, yy, label="")
        end
    end

    p
end
    

dibuja_voronoi (generic function with 1 method)

In [37]:
using Interact

In [145]:
n = 5
centros = [[rand(), rand()] for i in 1:n]
celdas_voronoi = voronoi_f(centros)

dibuja_voronoi(centros, celdas_voronoi, leg=true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip2302)" style="stroke:#00a9ad; stroke-width:1; stroke-opacity:1; fill:none" points="
 275.361,205.412 274.406,205.707 273.451,206.001 272.495,206.295 271.54,206.589 270.585,206.883 269.63,207.177 268.675,207.472 267.72,207.766 266.765,208.06 
 265.81,208.354 264.855,208.648 263.9,208.942 262.945,209.236 261.99,209.531 261.035,209.825 260.08,210.119 259.125,210.413 258.17,210.707 257.215,211.001 
 256.26,211.296 255.305,211.59 254.35,211.884 253.395,212.178 252.44,212.472 251.485,212.766 250.53,213.06 249.575,213.355 248.62,213.649 247.665,213.943 
 246.71,214.237 245.754,214.531 244.799,214.825 243.844,215.119 242.889,215.414 241.934,215.708 240.979,216.002 240.024,216.296 239.069,216.59 238.114,216.884 
 237.159,217.179 236.204,217.473 235.249,217.767 234.294,218.061 233.339,218.355 232.384,218.649 231.429,218.943 230.474,219.238 229.519,219.532 228.564,219.826 
 227.609,220.12 226.654,220.414 225.699,220.708 224.744,221.003 223.789,221.297 222.834,221.591 221.879,221.885 220.924,222.179 219.969,222.473 219.013,222.767 
 218.058,223.062 217.103,223.356 216.148,223.65 215.193,223.944 214.238,224.238 213.283,224.532 212.328,224.826 211.373,225.121 210.418,225.415 209.463,225.709 
 208.508,226.003 207.553,226.297 206.598,226.591 205.643,226.886 204.688,227.18 203.733,227.474 202.778,227.768 201.823,228.062 200.868,228.356 199.913,228.65 
 198.958,228.945 198.003,229.239 197.048,229.533 196.093,229.827 195.138,230.121 194.183,230.415 193.228,230.71 192.273,231.004 191.317,231.298 190.362,231.592 
 189.407,231.886 188.452,232.18 187.497,232.474 186.542,232.769 185.587,233.063 184.632,233.357 183.677,233.651 182.722,233.945 181.767,234.239 180.812,234.533 
 
 "/>
<polyline clip-path="url(#clip2302)" style="stroke:#ed5d92; stroke-width:1; stroke-opacity:1; fill:none" points="
 198.896,375.869 198.713,374.441 198.531,373.013 198.348,371.586 198.165,370.158 197.983,368.731 197.8,367.303 197.617,365.875 197.435,364.448 197.252,363.02 
 197.069,361.592 196.887,360.165 196.704,358.737 196.521,357.31 196.339,355.882 196.156,354.454 195.973,353.027 195.791,351.599 195.608,350.171 195.425,348.744 
 195.243,347.316 195.06,345.888 194.877,344.461 194.695,343.033 194.512,341.606 194.329,340.178 194.147,338.75 193.964,337.323 193.781,335.895 193.599,334.467 
 193.416,333.04 193.233,331.612 193.051,330.185 192.868,328.757 192.685,327.329 192.503,325.902 192.32,324.474 192.137,323.046 191.955,321.619 191.772,320.191 
 191.589,318.764 191.407,317.336 191.224,315.908 191.041,314.481 190.859,313.053 190.676,311.625 190.493,310.198 190.311,308.77 190.128,307.343 189.945,305.915 
 189.763,304.487 189.58,303.06 189.397,301.632 189.215,300.204 189.032,298.777 188.849,297.349 188.667,295.921 188.484,294.494 188.301,293.066 188.119,291.639 
 187.936,290.211 187.753,288.783 187.571,287.356 187.388,285.928 187.205,284.5 187.023,283.073 186.84,281.645 186.657,280.218 186.475,278.79 186.292,277.362 
 186.109,275.935 185.927,274.507 185.744,273.079 185.561,271.652 185.379,270.224 185.196,268.797 185.013,267.369 184.831,265.941 184.648,264.514 184.465,263.086 
 184.283,261.658 184.1,260.231 183.917,258.803 183.735,257.376 183.552,255.948 183.369,254.52 183.187,253.093 183.004,251.665 182.821,250.237 182.639,248.81 
 182.456,247.382 182.273,245.954 182.091,244.527 181.908,243.099 181.725,241.672 181.543,240.244 181.36,238.816 181.177,237.389 180.995,235.961 180.812,234.533 
 
 "/>
<polyline clip-path="url(#clip2302)" style="stroke:#c68125; stroke-width:1; stroke-opacity:1; fill:none" points="
 124.366,235.524 124.936,235.514 125.506,235.504 126.076,235.494 126.647,235.484 127.217,235.474 127.787,235.464 128.357,235.454 128.927,235.444 129.497,235.434 
 130.068,235.424 130.638,235.414 131.208,235.404 131.778,235.394 132.348,235

In [153]:
celdas_voronoi["1"]

1-element Array{Array,1}:
 Array{Float64,1}[[0.43164, 0.0], [0.358312, 0.546926]]

In [113]:
seg1, _ = intersecta_region(centros[1], centros[2])
seg2, _ = intersecta_region(centros[1], centros[3])
misma_region2(seg1[2], centros[1], recta(seg2...))

true

In [111]:
seg1[1]

2-element Array{Float64,1}:
 0.328585
 1.0     

In [103]:
seg1[1][1]

2-element Array{Float64,1}:
 0.328585
 1.0     

In [61]:
celdas_voronoi["1"]

1-element Array{Array,1}:
 Any[[0.248195, 1.0], [1.0, 0.225845]]

In [38]:
@manipulate for i in 1:4
    dibuja_voronoi(centros, celdas_voronoi, leg=true)
    scatter!(arr_to_xy(celdas_voronoi["1"][i]), label="")
end

Interact.Options{:SelectionSlider,Any}(1: "input" = 2 Any , "i", 2, "2", 2, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("1"=>1,"2"=>2,"3"=>3,"4"=>4), Dict{Any,Any}(Pair{Any,Any}(4, "4"),Pair{Any,Any}(2, "2"),Pair{Any,Any}(3, "3"),Pair{Any,Any}(1, "1"))), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip4802)" style="stroke:#00a9ad; stroke-width:1; stroke-opacity:1; fill:none" points="
 321.471,133.164 321.165,134.093 320.859,135.023 320.553,135.953 320.248,136.883 319.942,137.813 319.636,138.743 319.33,139.673 319.024,140.602 318.719,141.532 
 318.413,142.462 318.107,143.392 317.801,144.322 317.495,145.252 317.19,146.182 316.884,147.111 316.578,148.041 316.272,148.971 315.966,149.901 315.661,150.831 
 315.355,151.761 315.049,152.691 314.743,153.62 314.437,154.55 314.132,155.48 313.826,156.41 313.52,157.34 313.214,158.27 312.908,159.2 312.603,160.129 
 312.297,161.059 311.991,161.989 311.685,162.919 311.379,163.849 311.074,164.779 310.768,165.709 310.462,166.638 310.156,167.568 309.85,168.498 309.545,169.428 
 309.239,170.358 308.933,171.288 308.627,172.218 308.321,173.147 308.016,174.077 307.71,175.007 307.404,175.937 307.098,176.867 306.792,177.797 306.487,178.727 
 306.181,179.657 305.875,180.586 305.569,181.516 305.263,182.446 304.958,183.376 304.652,184.306 304.346,185.236 304.04,186.166 303.734,187.095 303.428,188.025 
 303.123,188.955 302.817,189.885 302.511,190.815 302.205,191.745 301.899,192.675 301.594,193.604 301.288,194.534 300.982,195.464 300.676,196.394 300.37,197.324 
 300.065,198.254 299.759,199.184 299.453,200.113 299.147,201.043 298.841,201.973 298.536,202.903 298.23,203.833 297.924,204.763 297.618,205.693 297.312,206.622 
 297.007,207.552 296.701,208.482 296.395,209.412 296.089,210.342 295.783,211.272 295.478,212.202 295.172,213.131 294.866,214.061 294.56,214.991 294.254,215.921 
 293.949,216.851 293.643,217.781 293.337,218.711 293.031,219.64 292.725,220.57 292.42,221.5 292.114,222.43 291.808,223.36 291.502,224.29 291.196,225.22 
 
 "/>
<polyline clip-path="url(#clip4802)" style="stroke:#ed5d92; stroke-width:1; stroke-opacity:1; fill:none" points="
 312.359,326.091 312.192,325.295 312.025,324.5 311.858,323.704 311.691,322.909 311.524,322.113 311.357,321.318 311.19,320.522 311.024,319.727 310.857,318.931 
 310.69,318.135 310.523,317.34 310.356,316.544 310.189,315.749 310.022,314.953 309.855,314.158 309.688,313.362 309.521,312.567 309.355,311.771 309.188,310.975 
 309.021,310.18 308.854,309.384 308.687,308.589 308.52,307.793 308.353,306.998 308.186,306.202 308.019,305.406 307.852,304.611 307.685,303.815 307.519,303.02 
 307.352,302.224 307.185,301.429 307.018,300.633 306.851,299.838 306.684,299.042 306.517,298.246 306.35,297.451 306.183,296.655 306.016,295.86 305.849,295.064 
 305.683,294.269 305.516,293.473 305.349,292.678 305.182,291.882 305.015,291.086 304.848,290.291 304.681,289.495 304.514,288.7 304.347,287.904 304.18,287.109 
 304.014,286.313 303.847,285.517 303.68,284.722 303.513,283.926 303.346,283.131 303.179,282.335 303.012,281.54 302.845,280.744 302.678,279.949 302.511,279.153 
 302.344,278.357 302.178,277.562 302.011,276.766 301.844,275.971 301.677,275.175 301.51,274.38 301.343,273.584 301.176,272.789 301.009,271.993 300.842,271.197 
 300.675,270.402 300.509,269.606 300.342,268.811 300.175,268.015 300.008,267.22 299.841,266.424 299.674,265.628 299.507,264.833 299.34,264.037 299.173,263.242 
 299.006,262.446 298.839,261.651 298.673,260.855 298.506,260.06 298.339,259.264 298.172,258.468 298.005,257.673 297.838,256.877 297.671,256.082 297.504,255.286 
 297.337,254.491 297.17,253.695 297.003,252.9 296.837,252.104 296.67,251.308 296.503,250.513 296.336,249.717 296.169,248.922 296.002,248.126 295.835,247.331 
 
 "/>
<polyline clip-path="url(#clip4802)" style="stroke:#c68125; stroke-width:1; stroke-opacity:1; fill:none" points="
 367.071,307.758 366.518,307.943 365.966,308.129 365.413,308.314 364.86,308.499 364.308,308.684 363.755,308.869 363.202,309.055 362.65,309.24 362.097,309.425 
 361.545,309.61 360.992,309.795 360.439,309.98 359.887,310.166 359.334,310.351 358.7

In [29]:
scatter(arr_to_xy(celdas_voronoi["1"][3]), ylims=(0,1.5), xlims=(0,1.5))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 
 
 
 
 
 
 
 
 y1

In [57]:
for key in keys(celdas_voronoi)
    println(celdas_voronoi[key])
end

Array[Any[[0.0, 0.147255], [0.799845, 1.0]]]
Array[Any[[0.0, 0.147255], [0.799845, 1.0]]]
Array[Any[[1.0, 0.353806], [0.176456, 0.0]]]


In [53]:
celdas_voronoi.keys()

LoadError: [91mMethodError: objects of type Array{String,1} are not callable
Use square brackets [] for indexing an Array.[39m

In [35]:
n_centros = 3
centros = [[rand(), rand()] for i in 1:n_centros]
celdas_voronoi = voronoi_final(centros)

dibuja_voronoi(centros, celdas_voronoi)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip6002)" style="stroke:#c271d2; stroke-width:1; stroke-opacity:1; fill:none" points="
 124.366,322.259 127.307,319.123 130.249,315.988 133.19,312.852 136.131,309.716 139.072,306.58 142.014,303.444 144.955,300.308 147.896,297.173 150.838,294.037 
 153.779,290.901 156.72,287.765 159.662,284.629 162.603,281.493 165.544,278.358 168.486,275.222 171.427,272.086 174.368,268.95 177.309,265.814 180.251,262.678 
 183.192,259.542 186.133,256.407 189.075,253.271 192.016,250.135 194.957,246.999 197.899,243.863 200.84,240.727 203.781,237.592 206.723,234.456 209.664,231.32 
 212.605,228.184 215.546,225.048 218.488,221.912 221.429,218.777 224.37,215.641 227.312,212.505 230.253,209.369 233.194,206.233 236.136,203.097 239.077,199.962 
 242.018,196.826 244.96,193.69 247.901,190.554 250.842,187.418 253.783,184.282 256.725,181.146 259.666,178.011 262.607,174.875 265.549,171.739 268.49,168.603 
 271.431,165.467 274.373,162.331 277.314,159.196 280.255,156.06 283.197,152.924 286.138,149.788 289.079,146.652 292.02,143.516 294.962,140.381 297.903,137.245 
 300.844,134.109 303.786,130.973 306.727,127.837 309.668,124.701 312.61,121.565 315.551,118.43 318.492,115.294 321.434,112.158 324.375,109.022 327.316,105.886 
 330.257,102.75 333.199,99.6146 336.14,96.4787 339.081,93.3429 342.023,90.2071 344.964,87.0712 347.905,83.9354 350.847,80.7995 353.788,77.6637 356.729,74.5278 
 359.671,71.392 362.612,68.2562 365.553,65.1203 368.494,61.9845 371.436,58.8486 374.377,55.7128 377.318,52.577 380.26,49.4411 383.201,46.3053 386.142,43.1694 
 389.084,40.0336 392.025,36.8978 394.966,33.7619 397.908,30.6261 400.849,27.4902 403.79,24.3544 406.732,21.2185 409.673,18.0827 412.614,14.9469 415.555,11.811 
 
 "/>
<polyline clip-path="url(#clip6002)" style="stroke:#ac8d18; stroke-width:1; stroke-opacity:1; fill:none" points="
 124.366,322.259 127.307,319.123 130.249,315.988 133.19,312.852 136.131,309.716 139.072,306.58 142.014,303.444 144.955,300.308 147.896,297.173 150.838,294.037 
 153.779,290.901 156.72,287.765 159.662,284.629 162.603,281.493 165.544,278.358 168.486,275.222 171.427,272.086 174.368,268.95 177.309,265.814 180.251,262.678 
 183.192,259.542 186.133,256.407 189.075,253.271 192.016,250.135 194.957,246.999 197.899,243.863 200.84,240.727 203.781,237.592 206.723,234.456 209.664,231.32 
 212.605,228.184 215.546,225.048 218.488,221.912 221.429,218.777 224.37,215.641 227.312,212.505 230.253,209.369 233.194,206.233 236.136,203.097 239.077,199.962 
 242.018,196.826 244.96,193.69 247.901,190.554 250.842,187.418 253.783,184.282 256.725,181.146 259.666,178.011 262.607,174.875 265.549,171.739 268.49,168.603 
 271.431,165.467 274.373,162.331 277.314,159.196 280.255,156.06 283.197,152.924 286.138,149.788 289.079,146.652 292.02,143.516 294.962,140.381 297.903,137.245 
 300.844,134.109 303.786,130.973 306.727,127.837 309.668,124.701 312.61,121.565 315.551,118.43 318.492,115.294 321.434,112.158 324.375,109.022 327.316,105.886 
 330.257,102.75 333.199,99.6146 336.14,96.4787 339.081,93.3429 342.023,90.2071 344.964,87.0712 347.905,83.9354 350.847,80.7995 353.788,77.6637 356.729,74.5278 
 359.671,71.392 362.612,68.2562 365.553,65.1203 368.494,61.9845 371.436,58.8486 374.377,55.7128 377.318,52.577 380.26,49.4411 383.201,46.3053 386.142,43.1694 
 389.084,40.0336 392.025,36.8978 394.966,33.7619 397.908,30.6261 400.849,27.4902 403.79,24.3544 406.732,21.2185 409.673,18.0827 412.614,14.9469 415.555,11.811 
 
 "/>
<polyline clip-path="url(#clip6002)" style="stroke:#00a9ad; stroke-width:1; stroke-opacity:1; fill:none" points="
 488.424,247.063 485.395,248.364 482.367,249.665 479.338,250.966 476.31,252.267 473.281,253.568 470.253,254.869 467.224,256.171 464.196,257.472 461.167,258.773 
 458.139,260.074 455.111,2

In [164]:
celdas_voronoi

Dict{String,Array{Array,1}} with 3 entries:
  "1" => Array[]
  "2" => Array[]
  "3" => Array[]

In [ ]:
function intersecta_planos(H)
    
    n = length(H)
    n_2 = Int(ceil(n/2))
    
    if n == 1
        C = H[1]
    else
        H1 = H[1:n_2]
        H2 = H[n_2+1:end]
        C1 = intersecta_planos(H1)
        C2 = intersecta_planos(H2)
        
    
    end

In [639]:
a = [[1,2], [2,3], [4,3],[3,4], [6,5], [6,7]]
n = length(a)

6

# Algoritmo Divide y Vencerás

El algoritmo de divide y vencerás en el caso de polígonos de Voronoi, es similar al de fuerza bruta, salvo que previamente tienes que dividir adecuadamente los centros en subregiones. Para esto, el primer paso es ordenar los puntos, de tal forma que en cada subregión geométrica haya el mismo número de puntos. Entonces, se calcula usando "fuerza bruta" los polígonos de Voronoi en cada región y se identifica los polígonos que se encuentran en la frontera. Como siguiente paso, se calcula los polígonos de Voronoi de los centros que estan en la frontera, para esto, se hacen sub-regiones "trasladadas" que incluyan a los centros que se encuentran en las fronteras en común además de los centros que sean vecinos de estos (por ejemplo, si la región A y B son frontera, se hará una nueva región AB, que contenga a los centros que de A que son frontera con B y sus vecinos, además de los centros de B que son frontera con A y sus vecinos). Esto deja aún algunos centros "esquinas", es decir, que las celdas que forman tienen vértices sobre alguna de las subregiones. Estos centros formarán varias celdas posibles, dependiendo de la región con la que se haya tomado sus vecinos. Tomando la unión de todos los vecinos y obteniendo el diagrama de Voronoi usando esos vecinos, se puede completar el diagrama usando directamente la intersección de las mediatrices (ya se sabe quienes son los vecinos). 

En el algoritmo, puede suceder que no tengamos suficientes vecinos que rodeen a los centros que están en contacto con las fronteras, por ejemplo, si hay un solo centro en la región. Si esto pasa, se requiere un paso más, extendiendo la región a todas las regiones vecinas. El proceso de extención tiene que repetirse hasta que haya algún centro que esté "completo", es decir, que todas sus fronteras sean con otros centros, a menos que la subregión esté justamente en una frontera, en cuyo caso todas sus fronteras deben ser con centros o la frontera de la caja en sí. 

[12] Haz una función que separe adecuadamente los puntos en $n$ subregiones. Trata de pensar "heurísticamente" cómo minimizar el número de puntos que estén en contacto con la frontera. Por supuesto, no hay "La" estrategia, depende de la topología de los puntos, pero hay estrategias que son típicamente malas y otroas que son típicamente mejores. 

[13] Haz una función que aplicando lo de fuerza bruta encuentre los polígonos de Voronoi de los centros que no estén en contacto con las fronteras y a su vez encuentre qué centros están en contacto con qué fronteras. Además debe arrojar una lista de los centros que son vecinos a cada uno de los diferentes centros en cada subregión. 

[14] Utilizando el problema anterior, haz una función que obtenga las subregiones trasladadas, que incluya a los elementos que están en cada frontera, además de sus vecinos. 

[15] Haz una función que desarrolle el algoritmo "Divide y Vencerás" Para una región finita dada (un cuadrado, por ejemplo)

[16] Haz una función con el mismo algoritmo, pero esta vez considerando condiciones periódicas a la frontera. 

***Nota: la función debe arrojar el teselado de Voronoi para ser graficado (los vértices de cada polígono de forma ordenada) y qué centros son vecinos de qué otros centros. También sería deseable que arroje el tamaño de las fronteras entre vecinos. ***


In [640]:
s = Int(ceil(n/2))

3

[17] Mide la complejidad computacional del algoritmo para el caso de condiciones periódicas a la frontera. 

[18] ¿Podrías generalizar este algoritmo a ND? ¿Qué requieres? 

[extra] Generaliza el algoritmo a ND.

# Algoritmo de Fortune

El segundo algoritmo que veremos, utiliza la idea de ir construyendo el teselado con una cantidad restringida de centros. Después de todo, si ya tenemos quienes son los vecinos de algún centro, ese centro queda descartado en la búsqueda, así por ejemplo, en el algoritmo de fuerza bruta, una mejora habría sido que una vez encontrado uno de los polígonos, descartáramos su centro en la búsqueda, de tal forma que para el segundo centro tendríamos $N-2$ posibles vecinos, pero además, sabemos que para cada uno de los vecinos de ese centro, habría 3 vecinos que ya estarían determinados, uno de ellos el centro mismo, los otros 2 serían los que formaran aristas vecinas en el polígono. 

La idea de Fortune fue ligeramente diferente. Pensemos que tenemos una recta que barre de arriba hacia abajo los centros de nuestros polígonos. De esta forma (similar al algoritmo divide y vencerás), la cantidad de puntos con la que "lidiamos" se ve reducida. El problema es que quizá algunos de los puntos que requerimos para formar algunos polígonos están debajo de la linea de barrido, igual que en el caso del algoritmo "divide y vencerás" con los puntos que colindan con la frontera. Podemos entonces usar la misma estrategia, encontrar los polígonos y si su frontera es con la linea de barrido, entonces esperar a que baje más la linea de barrido, apareciendo nuevos puntos. Si sus fronteras son todas con cetros, entonces "descartamos" de la lista esos puntos. 

El punto es ¿cómo saber qué tanto podemos bajar la linea de barrrido? Para esto, notemos que los puntos que distan lo mismo del centro que de la linea de barrido, son parábolas. La intersección entre dos parábolas correspondientes a dos centros que sean vecinos, será un punto que diste lo mismo de cada uno de los centros, es decir, un punto de la frontera. Cuando la linea de barrido llega a un nuevo centro, su parábola correspondiente será una semirrecta perpendicular a la linea de barrido y que pasa por el centro en cuestión. 

Pensemos por ejemplo, la linea llega al segundo centro, entonces, ya habrá una parábola y la semirrecta asociada al segundo centro intersectará a la parábola en un punto. Este punto tendría que ser parte de la frontera entre ambos centros. Mientras baja la linea de barrido, la segunda parábola se forma, generando un "hueco" en la primera. El segmento que une los extremos del hueco, serán parte de la frontera entre ambos centros. Ahora pensemos quela recta barredora, llega a un tercer centro. Esto abrirá nuevamente un "hueco" en alguna de las dos parábolas asociadas al centro 1 o al 2. Ambos huecos seguirán creciendo en ambas direcciones, hasta que se intersecten entre ellos. Entonces estos se "fusionarán". El punto en el que se fusionan, tendría que ser un vértice de nuestro polígono. 

La pregunta ahora es ¿cuánto tenemos que bajar nuestra recta para que ambos huecos se fusionen? El vértice que forma la fusión de huecos tiene que ser equidistante a cada uno de los 3 centros, es decir, es el centro de un círculo que pasa por los tres centros de los polígonos de Voronoi. Y entonces ¿a qué punto del barrido corresponde la fusión de huecos? La recta de barrido genera 3 parábolas que se intersectan en un punto, es decir, la distancia de ese punto a cada uno de los centros es la misma, pero también es la misma que la distancia a la recta, lo que se traduce en que la recta toca tangencialmente al círculo por el que pasan los 3 centros. Así, un vértice aparecerá cuando la recta de barrido pase por el polo inferior del círculo formado por los 3 centros y el vértice que aparece es el centro del círculo. 

Con esto generamos una forma de recorrer la recta de barrido, la bajaremos sobre una lista de puntos ordenados, los cuales contienen a los centros y a los polos de los círculos formados por 3 centros cuyas parábolas sean vecinas. Cuando llegamos a un nuevo centro, generamos polos y cuando llegamos a un polo generamos vértices de los polígonos de Voronoi. 

Sólo falta un detalle. A veces los polos pueden ser falsas alarmas, pues en el procesos de bajar la recta, aparecen nuevas parábolas y eso podría hacer que 3 parábolas que originalmente eran vecinas, dejan de serlo. Por lo tanto, cada vez que se llega a un nuevo polo, hay que verificar que sea uno real, es decir, que los centros a los cuales está asociado, siguen siendo centros vecinos, en el sentido de que sus parábolas sean vecinas. 

[19] Haz una función que calcule el centro de un círculo y su polo inferior, dados 3 puntos por los que pase. 

In [51]:
function calcula_centro(p1, p2, p3)
    
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    A = [x1^2+y1^2 y1 1; x2^2+y2^2 y2 1; x3^2+y3^2 y3 1]
    B = [x1 x1^2+y1^2 1; x2 x2^2+y2^2 1; x3 x3^2+y3^2 1]
    C = [x1 y1 1; x2 y2 1; x3 y3 1]
    h = det(A) / (2*det(C))
    k = det(B) / (2*det(C))
    
    r = norm(p1 - [h, k])
    
    [h, k], [h, k-r]
    
end

calcula_centro (generic function with 1 method)

In [91]:
A = []
B = [(x-pf) < δ for x in A]

0-element Array{Any,1}

In [90]:
!all(B)

false

In [65]:
puntos = [[rand(), rand()] for i in 1:3]
centro, polo = calcula_centro(puntos...)
scatter(arr_to_xy(puntos), ratio=:equal, xlims=(0, 2), ylims=(0,2))
scatter!([centro[1]], [centro[2]])
scatter!([polo[1]], [polo[2]])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 
 y2 
 
 
 
 
 y3

[20] Haz una función que ordene adecuadamente (por la coordenada $y$ de mayor a menor, y la $x$ también de mayor a menor) un conjunto de puntos (puedes usar sort). 

In [60]:
a = [4, 3, 2, 1]
ind = sortperm(a)
a[ind]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [61]:
function ordena(puntos)
    
    x, y = arr_to_xy(puntos)
    ind_x = sortperm(x)
    ind_y = sortperm(y)
    puntos = puntos[ind_x]
    puntos = puntos[ind_y]
    
    puntos
    
end

ordena (generic function with 1 method)

[21] Para hacer eficientemente el algoritmo, necesitarás tener una lista en forma de árbol binario. Esto quiere decir que un intervalo lo dividirás en subintervalos etiquedanto cada uno con un índice. Por ejemplo, el intervalo inicial puede ser [-Inf, Inf] y etiquetado con el índice 1 (Del primer centro). Después, al bajar la recta barredora, ese intervalo se cortará en 3 intervalos, [-Inf, x),[x,x],(x,Inf] y los índices 1,2,1, asociados a las regiones de la parábola 1 y el punto de intersección de la "parábola" 2. Haz una función que dado el conjunto de centros por encima de la recta barredora, el arreglo de índices y un centro dado (a la altura de la recta barredora), se agregue adecuadamente el índice del nuevo centro al conjunto de índices. 

In [12]:
function parabola(foco, l)
    
    a, b = foco
    y(x) = (1/(2*(b-l)))*((x-a)^2 + b^2 - l^2)
    y
    
end

parabola (generic function with 1 method)

In [ ]:
function binary_search_tree
    

In [ ]:
function handle_site_event(pi)
    
    if

In [164]:
a = [1,2,3]
b = [3,2,1]
ind_x = sortperm(a)
ind_y = sortperm(b)

3-element Array{Int64,1}:
 3
 2
 1

In [166]:
a = a[ind_x]
a = a[ind_y]

3-element Array{Int64,1}:
 3
 2
 1

In [167]:
a

3-element Array{Int64,1}:
 3
 2
 1

In [ ]:
function agrega_indice(sitios, sito, indices)
    
    x, y = sitio
    parabolas = [parabola(f, y) for f in sitios]
    ys = [p(x) for p in parabolas]
    ind = sortperm(ys)
    pj = sitios[ind[1]]
    

[22] Haz una función que dado un polo, el conjunto de puntos encima de la recta, y el árbol (conjunto de índices), revisa si el polo es realmente un "punto circular", es decir, que corresponde a 3 parábolas vecinas en el árbol. 

In [ ]:
function verifica_polo(polo, sitios, arbol)
    
    

[24] Haz una función que elimine adecuadamente los índices correspondientes a un punto circular. 

[25] Haz una función que dado un árbol, un conjunto de centros, un conjunto de puntos sobre los cual mover la recta y un centro (a la altura de la recta), agregue de forma adecuada un índice al arbol y quite el punto correspondiente del conjunto de puntos. 

[26] Haz una función que dado un árbol, un conjunto de centros, un conjunto de puntos, y un polo, quite los índices de forma adecuada, quite el punto correspondiente al conjunto de puntos y agregue un vértice a los polígonos de Voronoi correspondientes. 

[27] Haz una función que realice el algoritmo de Fortune para calcular los polígonos de Voronoi adecuadamente. Puedes usar algún algoritmo de convexhull para "ordenar" correctamente los puntos de cada polígono de Voronoi, o pensar cómo evitar hacerlo sin usar el convexhull (que por supuesto se puede). 

***Nota: Recuerda que aquí no mencioné cómo considerar una frontera, en principio no es necesaria, pero en ese caso tienes que asegurarte que los vértices de un poligono que no tengan más que un vecino, los escribas como un vector, es decir, asociando una dirección. Esta es la mejor forma, pues después puedes incrustar tus polígonos dentro de cualquier región, incluidas con condiciones periódicas. ***

[28] Mide la complegidad computacional del algoritmo de Fortune, y compara con Divide y Vencerás. ¿Tienen la misma complejidad? 

[29] Si tienen la misma complejidad, ¿qué algoritmo es más eficiente?


# Aplicaciones

Las aplicaciones de los polígonos de Voronoi son muchísimas y muy variadas, así que consideren investigar un poco al respecto para planear sus proyectos finales. 

Puesto que ya la tarea es lo bastante complicada y aceptaron tener una tarea 4 sobre generalizaciones de estos polígonos, decidí dejar la sección de aplicaciones para la siguiente tarea (que será más sencilla si logran hacer esta bien). Así que tienen 7 problemas menos que resolver por ahora...